In [20]:
import pandas as pd
from nltk.tag import pos_tag
import pymorphy2
from nltk import ngrams
from tqdm import tqdm
import re

In [4]:
itmo_data = pd.read_csv('Data/22.04.2020_21:43.csv', index_col=0)

In [5]:
itmo_data.head()

,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,clear_keywords
0,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятности, распределения, экстремума, вектор..."
1,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятности, распределения, экстремума, вектор..."
2,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряженности, вектор, электро..."
3,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряженности, вектор, электро..."
4,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,"семья, правонарушения, имущественные, правовог..."


In [10]:
key_words = itmo_data['clear_keywords'].apply(lambda x: str(x).split(', ')).values

In [12]:
key_words = [word for sent in key_words for word in sent]

In [15]:
morph = pymorphy2.analyzer.MorphAnalyzer()

In [63]:
itmo_data['clear_keywords'] = itmo_data['clear_keywords'].fillna(' ')

In [84]:
all_keywords = []

for key_words in tqdm(itmo_data['clear_keywords'].values):
    phrases = key_words.split(', ')
    tmp = []
    if phrases != []:
        for phrase in phrases:
            if len(phrase.split()) == 1:
                if re.search('[А-Яа-я]+', phrase):
                    if morph.parse(phrase)[0].tag.POS == 'NOUN':
                        tmp.append(morph.parse(phrase)[0].normalized[0])
                elif re.search('[A-Za-z]+', phrase):
                    if pos_tag([phrase])[0][1] == 'NN' or\
                    pos_tag([phrase])[0][1] == 'NNP' or pos_tag([phrase])[0][1] == 'NNPS':
                        tmp.append(phrase)

            elif len(phrase.split()) == 2:
                tokenized = []
                for word in phrase.split():
                    if re.search('[А-Яа-я]+', word):
                        tokenized.append(morph.parse(word)[0].tag.POS)
                    elif re.search('[A-Za-z]+', word):
                        tokenized.append(pos_tag([word])[0][1])

                tag_1 = tokenized[0]
                tag_2 = tokenized[1]
                
                if tag_1 != None and tag_2 != None:
                    if re.match(r'N', tag_1) and re.match(r'N', tag_2) or\
                    re.match(r'A', tag_1) and re.match(r'N', tag_2):
                        tmp.append(phrase)

            elif len(phrase.split()) == 3:
                tokenized = []
                for word in phrase.split():
                    if re.search('[А-Яа-я]+', word):
                        tokenized.append(morph.parse(word)[0].tag.POS)
                    elif re.search('[A-Za-z]+', word):
                        tokenized.append(pos_tag([word])[0][1])

                tag_1 = tokenized[0]
                tag_2 = tokenized[1]
                tag_3 = tokenized[2]
                if tag_1 != None and tag_2 != None and tag_3 != None:
                    if re.match(r'A', tag_1) and re.match(r'N', tag_2) and re.match(r'N', tag_3) or\
                    re.match(r'A', tag_1) and re.match(r'A', tag_2) and re.match(r'N', tag_3):
                        tmp.append(phrase)
        else:
            tmp.append(' ')

    all_keywords.append(', '.join(tmp))


100%|██████████| 53113/53113 [01:17<00:00, 687.49it/s] 


In [85]:
itmo_data['new_keywords'] = all_keywords

In [68]:
itmo_data.iloc[53109]['clear_keywords']

'научных документовструктура текста, среде labview, эксперимента программирования среде, экспериментальный набор данных, экспериментальных данных использованием, чтение визуализация сохранение, стандарты оформления, использованием gnuplot python'

URL                  
Название курса       
Содержание курса     
Дата сет             
Ключевые слова       
clear_keywords       
new_keywords         
Name: 53110, dtype: object

In [91]:
itmo_data['URL'].unique().shape

(966,)

In [92]:
data = pd.read_csv('Data/Itmo_with_tf_idf.csv', index_col=0)

In [93]:
data.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,Чистое содержание,keywords_tf-idf
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,элемент линейный алгебрытема линейный простран...,"{'час': 0.271, 'функция': 0.227, 'случайный': ..."
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,элемент линейный алгебрытема линейный простран...,"{'час': 0.271, 'функция': 0.227, 'случайный': ..."
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,механик механический колебание волныкинематика...,"{'колебание': 0.271, 'энергия': 0.244, 'вектор..."
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,механик механический колебание волныкинематика...,"{'колебание': 0.271, 'энергия': 0.244, 'вектор..."
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,такой право российский право правовой карта ми...,"{'право': 0.697, 'ответственность': 0.283, 'пр..."


In [94]:
data['key_words'] = itmo_data['new_keywords']

In [96]:
data = data[['SUBFIELDCODE', 'SUBFIELDNAME', 'DEGREE', 'SUBJECT', 'SEMESTER', 'START_YEAR', 'URL', 
            'Название курса', 'Содержание курса', 'Дата сет', 'Ключевые слова', 'key_words']]

In [103]:
data['all_keywords'] = itmo_data['clear_keywords']

<ipython-input-103-685b39d2bc35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['all_keywords'] = itmo_data['clear_keywords']


In [104]:
data.head()

,SUBFIELDCODE,SUBFIELDNAME,DEGREE,SUBJECT,SEMESTER,START_YEAR,URL,Название курса,Содержание курса,Дата сет,Ключевые слова,key_words,all_keywords
0,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,1,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","вероятности, распределения, экстремума, вектор..."
1,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Математика,2,2014,https://www.hse.ru/edu/courses/292700221,Математика,Элементы линейной алгебрыТема 1.1. линейные пр...,hse,,"вероятность, распределение, экстремум, сложной...","вероятности, распределения, экстремума, вектор..."
2,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,1,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","импульс, поток, напряженности, вектор, электро..."
3,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Физика,2,2014,https://www.hse.ru/edu/courses/292684119,Физика,Тема 1. Механика. Механические колебания и вол...,hse,,"импульс, поток, напряжённость, вектор, электро...","импульс, поток, напряженности, вектор, электро..."
4,09.03.01,"Вычислительные машины, комплексы, системы и сети",Академический бакалавр,Основы права,1,2014,https://online.edu.ru/public/course.xhtml?face...,Основы права,Что такое право? Российское право на правовой...,itmo,,"семья, правонарушение, правового регулирования...","семья, правонарушения, имущественные, правовог..."


In [110]:
data['key_words'] = data['key_words'].apply(lambda x : re.sub('ит', 'it', x) if re.search('[a-z]+ит[a-z]+', s) else x)

<ipython-input-110-cf87c490c7c0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['key_words'] = data['key_words'].apply(lambda x : re.sub('ит', 'it', x) if re.search('[a-z]+ит[a-z]+', s) else x)


In [111]:
data['key_words']

0        вероятность, распределение, экстремум, сложной...
1        вероятность, распределение, экстремум, сложной...
2        импульс, поток, напряжённость, вектор, электро...
3        импульс, поток, напряжённость, вектор, электро...
4        семья, правонарушение, правового регулирования...
                               ...                        
53108                                                     
53109    научных документовструктура текста, среде labv...
53110                                                     
53111    результативность, государственного управления ...
53112                                                     
Name: key_words, Length: 53113, dtype: object

In [115]:
all_keywords[-4]

'научных документовструктура текста, среде labview, стандарты оформления,  '

In [ ]:
change = {'документовструктура' : 'структура документов', 'научных документовструктура текста' : 'структура научных документов',
         'systemsinformation': 'systems information', 'statisticspopulation' : 'statistics population', 'цепейэлектрическая' : 'электрические цепи'б
         'processproject': 'process project', 'management1' : 'management', 'managementinvestment': 'management investment',
         'ffff' : '', 'r2классификация' : 'r2 классификация', 'деятельностипонятие' : 'понятие деятельности',
          'деятельностивиды' : 'виды деятельности', 'introduction1' : '', 'стартапаисточник' : 'стартап', 'лидерствовид' : 'лидерство',
          'аналитикаструктура' : 'структура аналитики', 'моделимультиферроики' : 'модели мультиферроики',
          'технологийблок' : 'блок технологий', 'клиентаот' : 'клиент аот', 'managementequivalent' :'management equivalent',
          'интервьюфокус' : 'интервью', 'информацииконтент' : 'информационный контент', 'исследованияпонятие' : '',
          'исследованиитеории' : ''
         }

In [118]:
from sklearn.feature_extraction.text import CountVectorizer

In [119]:
# Getting top n grams function
# Arguments:
#   - corpuse, we want to get ngrams from, number of words to output
#     and n gram range
#        corpus : pd.Series, number_of_words : int = None, n_grams : int = 1
# Rerutns:
#   - words frequencies list
#       (list)
def get_top_n_grams(corpus : pd.Series, number_of_words : int = None, n_grams : int = 1) -> list:
    # vectorizing corpus using Count vectorizer
    vec = CountVectorizer(ngram_range=(n_grams, n_grams)).fit(corpus)
    
    # getting bag of words
    bag_of_words = vec.transform(corpus)
    
    # getting words frequencies
    sum_words = bag_of_words.sum(axis=0) 
    
    # building list of words and their freqs
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    # sorting words freq list
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    return words_freq[:number_of_words]

In [180]:
change = {'документовструктура' : 'структура документов', 'научных документовструктура текста' : 'структура научных документов',
         'systemsinformation': 'systems information', 'statisticspopulation' : 'statistics population', 'цепейэлектрическая' : 'электрические цепи',
         'processproject': 'process project', 'management1' : 'management', 'managementinvestment': 'management investment',
         'ffff' : '', 'r2классификация' : 'r2 классификация', 'деятельностипонятие' : 'понятие деятельности',
          'деятельностивиды' : 'виды деятельности', 'introduction1' : '', 'стартапаисточник' : 'стартап', 'лидерствовид' : 'лидерство',
          'аналитикаструктура' : 'структура аналитики', 'моделимультиферроики' : 'модели мультиферроики',
          'технологийблок' : 'блок технологий', 'клиентаот' : 'клиент аот', 'managementequivalent' :'management equivalent',
          'интервьюфокус' : 'интервью', 'информацииконтент' : 'информационный контент', 'исследованияпонятие' : '',
          'исследованиитеории' : '', 'студентоввыступление': 'выступление студентов', 'тматрица' : 'матрица',
          'tariffstrategic' : 'tariff strategic', 'debateenvironment' : 'debate environment', 'ситуациейнаправления' : '',
          'structureidealism' : 'structure idealism','processmarkov' : 'Markov process', 'processpoisson' : 'Poisson process',
          'chaingaussian' : 'Gaussian chain', 'проектасоздание' : 'создание проекта', 'кубитовклассическая' : 'кубитов классическая',
          'presentationsguest' : 'presentations guest', 'сетейопределение' : 'определение сетей', 'zabbixустановка' : 'zabbix установка',
          'деятельностьпонятие' : 'понятие деятельности', 'заданиякомпозиция' : 'композиция', 'буклеттестирование' : '',
          'интерфейсастуденты': 'интерфейс', 'закладоккомпьютерные' : 'компьютерные закладки', 'поставокоптимизация' : 'оптимизация поставок',
          'гмувведение' : 'введение в гму', 'стэкнаследование' : 'стэк, наследование', 'мальтусэкономическая' : 'экономические взгляды Мальтуса',
          'discourselexicone' : 'lexicone', 'morphologysyntax' : 'morphology syntax', 'чтениеграмматика' : 'чтение, грамматика',
          'managementproject': 'management project', 'businessproduct' : 'business product', 'businessbusiness' : '',
          'проектахбизнес' : 'бизнес проекты', 'innovationprotection' : 'innovation protection', 'поставоктехнология' : 'технология поставок',
          'отношенияфункция' : 'функция отношения', 'эйконаланелинейные' : 'уравнение эйконала', 'интеграловосновные' : 'основные интегралы',
          'частицамидвижение': 'движение частиц', 'корпорациисистема' : 'система корпораций', 'этапасодержание': '',
          'инженериистратегия' : 'стратегия инженерии', 'компанияпринципы' : 'принципы компании','обеспечениясоздание' : 'создание обеспечения',
          'материаловедение' : 'ведение материалов', 'системераздел' : '', 'валентностипонятие' : 'понятие валентности',
          'pythonалгоритмы' : 'python алгоритмы', 'интернетработа' : 'работа в интернете', 'данныеконцепция' : 'концепция данных',
          '2вязкая' : 'вязкая', 'алгоритмывведение' : 'введение в алгоритмы', 'techniquesnaive' : 'naive techniques', 
          'selectionregression' : 'selection regression', 'industriestopic' : 'industries topic', 'stylesequilibrium' : 'styles equilibrium',
          'правапредмет' : 'предмет права', 'marketinvestment' : 'market investment', 'анализбюджетирование' : 'анализ бюджетирования',
          'тестовпрактические' : '', 'мышлениимышление': 'мышление', 'восприятииосновные' : 'основные восприятия',
          'перспективуоценка' : 'оценка перспективы', 'системыосновные' : '', 'типологияотношение' : 'типология отношений',
          'физикаобсуждение' : 'обсуждение физики', 'бизнесатематическая' : 'бизнес', 'аспектформулирование'  :'формулилование',
          'имуществаструктура' : 'структура имущества', 'капиталаструктура' : 'структура капитала', 'креативабренд' : 'креативный бренд',
          'investmentsvaluation' : 'investments valuation', 'discussionsocial' : 'social discussion', 'управлениясравнение' : 'сравнение уравление',
          'csdiscuss' : 'cs discuss', 'semanticspredicate' : 'semantics predicate', '1международное' : 'международное', 'излучателиплоские' : 'плоские излучатели',
          'мышлениеоперирование' : 'мышление, оперирование', 'проектамипонятие' : 'понятие проекта', 'памятиотличительные' : 'память',
          'сравнениеисточники' : 'источники сравнения', 'проектамиосновные' : '', 'otherclassification' : 'other classification',
          'проектаосновные' : 'основные проекты', 'visionfechner' : 'vision fechner', 'переходыфлуктуации' : 'переходы флуктуации',
          'modelsdynamic' : 'models dynamic', 'marketingsocial' : 'social marketing',  'развитияопределение' : 'определение развития',
          'медиарынокпонятие': 'понятие медиарынка', 'расчетоввизуализация' : 'визуализация расчетов', 'обеспеченияосновы' : 'основы обеспечения',
          'zeитvertreibтема' : 'zeitvertreib', 'системаходнофотонные' : 'фотон', 'персоналаработа' : 'работа персонала',
          'поискаусловие' : 'поиск условия', 'усилителиназначение' : 'назначение усилителей', 'компаниипонятие' : 'понятие компанияя',
          'телевиденияобработка' : 'обработка на телевидение', 'радиоуправлениятема' : 'радиоуправление','массивовалгоритм' : 'алгоритм с массивами',
          'программпараллельные' : 'параллельное программирование', 'сверхрешеткахспин' : 'решетка, спин', 
          'поведенияэтика' : 'этика поведения', 'направленийэтап' : '', 'managementhistory' : 'management history',
          'emergenceinternet' : 'emergence internet', 'methodscluster' : 'cluster methods', 'концентраторымост' : 'концентраторы, мост',
          'selectiondimension': 'dimention selection', 'treeslinear' : 'linear trees', 'analysismachine' : 'analysis machine',
          'rattleдерево' : 'rattle дерево', 'problemmarket' : 'market problem', 'controlprincipal' : 'control principal',
          'projectорганизация' : 'организация проекта', '2системы' : 'системы', 'свободылинейные' : '',
          'фильтровопределение' : 'определение фильтров', 'введениепредмет' : '', 'средсттехнико' : 'технические средства',
          'средствуправление' : 'средства управления', 'постнеклассической' : 'неклассический пост', 'разработкаистория' : 'разработка истории',
          'spssнабор' : 'spss набор', 'djangoнейросеть' : 'django нейросеть', 'структураопределение' : 'определение структуры',
          'исследованийчистые' : 'чистые исследования', 'modelsproject' : 'project models', 'lorawanособенность' : 'lorawan особенность',
          'уравнениевывод' : 'вывод уравнения', 'безопасностипонятие' : 'понятие безопасности', 'системыопределение' : 'определение системы',
          'классыобработка' : 'обработка классов', 'gamesequilibrium' : 'games equilibrium','компьютераосновные' : 'компьютер',
          'системмоделирование' : 'моделирование систем', 'анализасистемный' : 'системный анализ', 'courseintroduction' : '',
          'моделиметодология' : 'модели методологии', 'введениеосновная' : '', 'графовтеорем' : 'теоремы на графах', 
          'функциипроизводящие' : 'функции', 'карьереособенности' : 'особенности карьеры', 'введениевведение' : '', 
          'softwarecmt' : 'software cmt', 'внутрифирменное' : 'фирменное', 'сетипротоколы' : 'протоколы сети',
          'системинформационные' : 'информационные системы', 'statisticspopulation' : 'statistics population',
          'цепейэлектрическая' : 'электрическая цепь', 'systemsinformation' : 'information systems',
          'раннесредневековой' : 'средневековой'
            
         }

In [184]:
for key, item in change.items():
    data['key_words'] = data['key_words'].apply(lambda x: re.sub(key, item, x))

<ipython-input-184-380d4b12a83e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['key_words'] = data['key_words'].apply(lambda x: re.sub(key, item, x))


In [190]:
data['key_words']

0        вероятность, распределение, экстремум, сложной...
1        вероятность, распределение, экстремум, сложной...
2        импульс, поток, напряжённость, вектор, электро...
3        импульс, поток, напряжённость, вектор, электро...
4        семья, правонарушение, правового регулирования...
                               ...                        
53108                                                     
53109    научных структура документов текста, среде lab...
53110                                                     
53111    результативность, государственного управления ...
53112                                                     
Name: key_words, Length: 53113, dtype: object

In [192]:
data.iloc[0]

SUBFIELDCODE                                                 09.03.01
SUBFIELDNAME         Вычислительные машины, комплексы, системы и сети
DEGREE                                         Академический бакалавр
SUBJECT                                                    Математика
SEMESTER                                                            1
START_YEAR                                                       2014
URL                          https://www.hse.ru/edu/courses/292700221
Название курса                                             Математика
Содержание курса    Элементы линейной алгебрыТема 1.1. линейные пр...
Дата сет                                                          hse
Ключевые слова                                                       
key_words           вероятность, распределение, экстремум, сложной...
all_keywords        вероятности, распределения, экстремума, вектор...
Name: 0, dtype: object

In [193]:
data.iloc[0]['key_words']

'вероятность, распределение, экстремум, сложной функции, переменных функции, линейных пространств векторы, линейных алгебраических уравнений, линейный коэффициент корреляции, случайная величина,  '

In [194]:
data.iloc[10]

SUBFIELDCODE                                                 09.03.01
SUBFIELDNAME         Вычислительные машины, комплексы, системы и сети
DEGREE                                         Академический бакалавр
SUBJECT                                           Физическая культура
SEMESTER                                                            2
START_YEAR                                                       2014
URL                 https://online.edu.ru/public/course.xhtml?face...
Название курса                                   Физическая культура 
Содержание курса     Тема 1. Общая концепция теории и методики физ...
Дата сет                                                         itmo
Ключевые слова      здоровый образ жизни студента, спорт, физиолог...
key_words           физическими упражнениями, физической культуры,...
all_keywords        физическими упражнениями, человека физиологиче...
Name: 10, dtype: object

In [195]:
data.iloc[10]['key_words']

'физическими упражнениями, физической культуры, здоровый образ жизни, центральная физической культуры, прикладной физической спорт, физическое воспитание,  '

In [197]:
data.iloc[23]

SUBFIELDCODE                                                 09.03.02
SUBFIELDNAME        Автоматизация и управление в образовательных с...
DEGREE                                         Академический бакалавр
SUBJECT                                              Иностранный язык
SEMESTER                                                            2
START_YEAR                                                       2014
URL                          https://www.hse.ru/edu/courses/301401516
Название курса                                       Иностранный язык
Содержание курса    Личность. Описание внешности, физических харак...
Дата сет                                                          hse
Ключевые слова                                                       
key_words           грамматика, коммуникативные humans living, язы...
all_keywords        грамматика, коммуникативные humans living, язы...
Name: 23, dtype: object

In [198]:
data.iloc[23]['key_words']

'грамматика, коммуникативные humans living, языковые вокабуляр nouns, социальные обсуждения, коммуникативных ситуациях общения,  '

In [199]:
data.to_csv('Data/itmo_23.04.2020_15:31.csv')